In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [2]:
training_df = pd.read_csv('~/train.csv', error_bad_lines=False)

In [3]:
training_df.shape

(404290, 6)

In [4]:
test_df = pd.read_csv('~/test.csv')

/Users/pranavhonrao/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



In [5]:
test_df.shape

(3563475, 3)

In [6]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
id              404290 non-null int64
qid1            404290 non-null int64
qid2            404290 non-null int64
question1       404289 non-null object
question2       404288 non-null object
is_duplicate    404290 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [7]:
feature_df =training_df['is_duplicate']

In [8]:
training_df =training_df.drop(columns=['is_duplicate'])

In [9]:
training_error1 =training_df['question1']

In [10]:
training_error2 =training_df['question2']

In [11]:
#training_error1.head()

In [12]:
#training_error2.head()

In [11]:
questions_training_df = training_df[['question1','question2']]

In [12]:
questions_test_df = test_df[['question1','question2']]

In [13]:
temp_check = pd.concat([questions_training_df, questions_test_df])

In [14]:
from nltk.corpus import stopwords
from nltk.stem.porter import *
import nltk
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pranavhonrao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pranavhonrao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
sw_dict={}
    
for i in stopwords.words('english'):
    sw_dict[i]=1
    



In [16]:
import re
from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
from stemming.porter2 import stem

lemi_words=[]
stem_words=[]
lmtzr = nltk.WordNetLemmatizer().lemmatize
ps = PorterStemmer()



def do_lemitise(word):
    old_word=word
    word = lmtzr(word)
    if word != old_word:
        lemi_words.append((old_word, word))
        
    return word
    
def do_stem(word):
    old_word=word
    word = stem(word)
    if word != old_word:
        stem_words.append((old_word, word))
    return word
        

def do_process_word(word):
    word = do_stem(word)
    word = do_lemitise(word)
    return word
    
    
    

def corpus_filteration(dfList):
    
    actual_list =[]
    

    
    item_num=0

    for k, v in dfList.iteritems():
        questions_corpus = re.sub('[^a-zA-Z]',' ', str(v)).lower().split()
        '''
        for word in questions_corpus:
            if word not in sw_dict.keys():
                #old_word=word
                word = lmtzr(word)
                #if word != old_word:
                    #lemi_words.append((old_word, word))
                questions_corpus.append(word)
        '''
                
                
                
            
        questions_corpus = [do_process_word(word) for word in questions_corpus if word not in sw_dict.keys()]
        
        actual_list.append(questions_corpus)
        item_num = item_num + 1
        if (item_num%1000000 == 0):
            print ("Done so far {} ".format(item_num) )
    
    return actual_list           
    
    

In [19]:
training_error1.head()

0    What is the step by step guide to invest in sh...
1    What is the story of Kohinoor (Koh-i-Noor) Dia...
2    How can I increase the speed of my internet co...
3    Why am I mentally very lonely? How can I solve...
4    Which one dissolve in water quikly sugar, salt...
Name: question1, dtype: object

In [17]:
training_error1_cf = corpus_filteration(training_error1)

In [21]:
#training_error1_cf[:10]

In [22]:
#lemi_words[:60]

In [23]:
#stem_words[:60]

In [18]:
training_error2_cf = corpus_filteration(training_error2)

In [19]:
temp_check_test_question1 = temp_check['question1']

In [20]:
test_question1_list1 = corpus_filteration(temp_check_test_question1)

Done so far 1000000 
Done so far 2000000 
Done so far 3000000 


KeyboardInterrupt: 

In [ ]:
temp_check_test_question2 = temp_check['question2']

In [ ]:
test_question2_list1 = corpus_filteration(temp_check_test_question2)

In [29]:
#test_check_01 = test_question1_list1[:12]

In [30]:
#test_check_01

In [31]:
#test_check_02= test_question2_list1[:12]

In [32]:
#test_check_02

In [ ]:
test_question_list= test_question1_list1 + test_question2_list1

In [34]:
from gensim.models import Word2Vec
model = Word2Vec(test_question_list, min_count=1,size=200)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
#print(words)
# access vector for one word
#print(model['hack'])
# save model
model.save('model.bin')
# load model
#new_model = Word2Vec.load('model.bin')
#print(new_model)

Word2Vec(vocab=82404, size=200, alpha=0.025)


In [35]:
#print(model['step'])

In [36]:

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
#filename = 'GoogleNews-vectors-negative300.bin'
#import pyemd
#word2vec_model = Word2Vec.load(filename)
word2vec_model = KeyedVectors.load('model.bin')
word2vec_model.init_sims(replace=True)


In [37]:
'''
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
model.init_sims(replace=True)
'''
    
   


"\nfrom gensim.scripts.glove2word2vec import glove2word2vec\nfrom gensim.models import KeyedVectors\nglove_input_file = 'glove.6B.100d.txt'\nword2vec_output_file = 'glove.6B.100d.txt.word2vec'\nglove2word2vec(glove_input_file, word2vec_output_file)\nfilename = 'glove.6B.100d.txt.word2vec'\nmodel = KeyedVectors.load_word2vec_format(filename, binary=False)\nmodel.init_sims(replace=True)\n"

In [38]:
'''
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
model.init_sims(replace=True)
'''


"\nfilename = 'glove.6B.100d.txt.word2vec'\nmodel = KeyedVectors.load_word2vec_format(filename, binary=False)\nmodel.init_sims(replace=True)\n"

In [39]:
from pyemd import emd
def distance_calculation(d1,d2):
        i=j=0
        distance_list=[]
        len_train_01 =len(d1)
        len_train_02 =len(d2)
        item_num=0
        
        
        while (i<len_train_01 and j<len_train_02):
            distance_questions = word2vec_model.wv.wmdistance(d1[i], d2[j])
            #f.write(str(distance_questions))
            distance_list.append(distance_questions)
            i=i+1
            j=j+1
            item_num = item_num + 1
            if (item_num%100000 == 0):
                print ("Done so far {} ".format(item_num) )
        
        training_df['is_duplicate_01']=distance_list
        return training_df
        #return distance_list

In [40]:
distance_calculation(training_error1_cf,training_error2_cf)

Done so far 100000 
Done so far 200000 
Done so far 300000 
Done so far 400000 


,id,qid1,qid2,question1,question2,is_duplicate_01
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.189705
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.820117
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.611369
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,1.342272
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,1.006852
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",0.646358
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,1.382144
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,0.468418
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0.000000
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0.653112


In [41]:
 #training_df.head()

In [42]:
#feature_df[:10]

In [43]:

item_num =0
for i, row in training_df['is_duplicate_01'].iteritems():
    if row == 0.000000:
        training_df.set_value(i,'is_duplicate_01',1)
    elif (row >= 0.28 and row <= 0.72):
        training_df.set_value(i,'is_duplicate_01',1)
    else:
        training_df.set_value(i,'is_duplicate_01',0)
    item_num = item_num + 1
    if (item_num%100000 == 0):
        print ("Done so far {} ".format(item_num) )
        
   


/Users/pranavhonrao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/Users/pranavhonrao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead

/Users/pranavhonrao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning:

set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



Done so far 100000 
Done so far 200000 
Done so far 300000 
Done so far 400000 


In [44]:
#training_df.head()

In [45]:
#training_df['is_duplicate_01'][:20]
#training_df['is_duplicate_01']=training_df[int(training_df['is_duplicate_01'])]
training_df['is_duplicate_01'] = training_df['is_duplicate_01'].astype(int)

In [46]:
#training_df.head()

In [47]:
#len(training_df)

In [48]:
from sklearn.metrics import confusion_matrix,accuracy_score

y_actu = feature_df
y_pred = training_df['is_duplicate_01']
#cf_ma = confusion_matrix(y_actu, y_pred)
a_score=accuracy_score(y_actu, y_pred)

In [49]:
cf_ma = confusion_matrix(y_actu, y_pred)
cf_ma

array([[153427, 101600],
       [ 38702, 110561]])

In [50]:
#cf_ma
a_score

0.6529669296791907

In [51]:
#accuracy_score =67.30 when dimension=100
#chaging to 300 now

In [52]:
def avg_length_corpus(training_error1_cf):
    
    min_length=0
    max_length=len(training_error1_cf[0])
    
    for i in range(0,len(training_error1_cf)):
        if len(training_error1_cf[i]) <=min_length :
            min_length= len(training_error1_cf[i])
        
    
    for i in range(1,len(training_error1_cf)):
        if len(training_error1_cf[i]) > max_length:
            max_length= len(training_error1_cf[i])
        
    
    avg_length= (min_length + max_length)//2
    
    
    return min_length,max_length,avg_length



In [53]:
avg_question1_training_set= avg_length_corpus(training_error1_cf)

In [54]:
avg_question2_training_set= avg_length_corpus(training_error2_cf)

In [55]:
print(avg_question1_training_set)
print(avg_question2_training_set)

(0, 53, 26)
(0, 97, 48)


In [56]:
def len_trainging_questions_array(training_error1_cf):
    questions_length_array =[]
    for i in training_error1_cf:
        questions_length_array.append(len(i))
        
    return questions_length_array
    

In [57]:
question_1_length =len_trainging_questions_array(training_error1_cf)

In [58]:
question_2_length =len_trainging_questions_array(training_error2_cf)

In [59]:

import plotly.plotly as py
import plotly.graph_objs as go


trace = go.Box(
    y= question_2_length,
    name='Mean & SD',
    marker=dict(
        color='rgb(10, 140, 208)',
    ),
    boxmean='sd'
)
data = [trace]
#py.iplot(data)
plotly.offline.plot(data)


'file:///Users/pranavhonrao/anaconda3/temp-plot.html'

In [60]:
def duplicate_entries(input_array):
    
    final_result=[]
    new_array=[]
    unseen=set()
    count=0
    for i in input_array:
        for j in i:
                 if j not in unseen:
                    unseen.add(j)
                    final_result.append(model[j])
                    #final_result.append(j)
        if len(final_result) >9:
            final_result=final_result[0:9]
        else:
            array_length=len(final_result)
            while array_length <9:
                final_result.append(np.zeros(200))
                array_length=array_length +1
                    
        new_array.append(final_result)
        final_result=[]
            
    
    return new_array
    
            



In [61]:
vector_training_set_question1 =duplicate_entries(training_error1_cf)

/Users/pranavhonrao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [62]:
#print(type(vector_training_set_question1))

In [63]:
vector_training_set_question2 =duplicate_entries(training_error2_cf)


            
        

/Users/pranavhonrao/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [64]:
#vector_training_set_question2

In [65]:
a = np.array(vector_training_set_question1)
b = np.array(vector_training_set_question2)





In [66]:
result=np.column_stack((a,b))

In [67]:
#len(result)

404290

/Users/pranavhonrao/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [69]:
#in_dim = 3600

In [70]:
y=feature_df

In [71]:
X=result

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=42)

In [77]:
#KNN

In [ ]:

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  


In [ ]:
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=3)  
classifier.fit(X_train, y_train)  

In [ ]:
y_pred = classifier.predict(X_test) 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))

In [ ]:
len(X_test)

In [ ]:
#score

In [ ]:

'''
print('Test score:', score[0])
print('Test accuracy:', score[1])
'''

In [ ]:
#print(history.history.keys())

In [ ]:
'''
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
'''

In [ ]:
'''
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
'''

In [ ]:
#predictions = model.predict(X_test)
#print('First prediction:', predictions[0])

In [ ]:
#KNN